In [83]:
path = '/Users/dpwe/Documents/Arduino/libraries/Adafruit_GFX_Library/Fonts/FreeMono9pt7b.h'
#path = '/Users/dpwe/Documents/Arduino/libraries/Adafruit_GFX_Library/Fonts/CalBlk36.h'

def parse_gfx_font_file(path):
    bracelevel = 0
    vals = [[], []]
    valpart = 0
    with open(path, 'rt') as f:
        for l in f:
            while l:
                l = l.strip()
                if not l or l[0] == '/':
                    break
                if '//' in l or '/*' in l:
                    # Strip trailing comment.  Does not handle multi-line /*..*/.
                    comment = '//' if '//' in l else '/*'
                    pos = l.index(comment)
                    l = l[:pos].strip()
                    continue
                if l[0] in '{},':
                    # Update brace level or ignore comma after brace block.
                    bracelevel += [1, -1, 0]['{},'.index(l[0])]
                    l = l[1:]
                    continue
                if '{' in l or '}' in l:
                    # Parse up until next brace.
                    brace = '{' if '{' in l else '}'
                    pos = l.index(brace)
                    payload = l[:pos].strip()
                    l = l[pos:]
                else:
                    payload = l.strip()
                    l = ''
                if bracelevel and payload and payload[0] != '/':
                    radix = 16 if '0x' in payload else 10
                    payload_vals = [int(v, radix) for v in payload.split(',') if v]
                    if bracelevel == 1:
                        vals[valpart].extend(payload_vals)
                    else:
                        vals[valpart].append(payload_vals)
            if len(vals[valpart]) and bracelevel == 0:
                valpart += 1
                if valpart == 2:
                    break
    return vals

vals = parse_gfx_font_file(path)

print(len(vals), len(vals[0]), len(vals[1]))

2 844 95


In [84]:
print(vals[1][-2])
print(vals[1][-1])
# {offset, width, height, advance cursor, x offset, y offset}


[836, 3, 13, 11, 4, -10]
[841, 7, 3, 11, 2, -6]


In [130]:
c = 0x32 - 32
bytes = vals[0][vals[1][c][0] : vals[1][c+1][0]]
width = vals[1][c][1]
height = vals[1][c][2]
print(vals[1][c])

def unpack_bits(bytes, width, height):
    """Return bytes in width-bit chunks."""
    pos = 0
    bits_left = 8
    this_byte = bytes[pos]
    result = [0 for _ in range(height)]
    for h in range(height):
        for b in range(width):
            result[h] = 2 * result[h] + (this_byte > 127)
            bits_left -= 1
            if bits_left:
                this_byte = (2 * this_byte) & 0xFF
            else:
                pos += 1
                if pos < len(bytes):
                    this_byte = bytes[pos]
                bits_left = 8
    return result

def show_bitmap(bytes, bitwidth=8):
    for b in bytes:
        print(''.join([[' ', '*']['01'.index(c)] for c in '{{:0{:d}b}}'.format(bitwidth).format(b)]))

fvals = unpack_bits(bytes, width, height)
show_bitmap(fvals, width)
print(bytes)
print(fvals)

[107, 7, 11, 11, 2, -10]
  ***  
 *   * 
*     *
      *
      *
     * 
    *  
   *   
 **    
*      
*******
[56, 138, 8, 16, 32, 130, 8, 97, 3, 248]
[28, 34, 65, 1, 1, 2, 4, 8, 48, 64, 127]


In [131]:
print(1234 & 0x7f)

82


In [137]:
def pack_bits(vals, width):
    """Return width bits from each val packed into bytes."""
    bytes = []
    this_byte = 0
    bits_to_go = 8
    for v in vals:
        for b in range(width):
            this_byte *= 2
            bitval = 1 if (v & (1 << (width - b - 1))) else 0
            if bitval:
                this_byte += 1
            bits_to_go -= 1
            if bits_to_go == 0:
                bytes.append(this_byte)
                this_byte = 0
                bits_to_go = 8
    if bits_to_go < 8:
        bytes.append(this_byte << bits_to_go)
    return bytes

b = pack_bits(fvals, width)
print(len(b), b)

10 [56, 138, 8, 16, 32, 130, 8, 97, 3, 248]


In [138]:
def parse_simple_font_file(path):
    """Parse one-byte-per-row file written by https://eleif.net/HD44780.html."""
    bracelevel = 0
    vals = []
    with open(path, 'rt') as f:
        for l in f:
            while l:
                l = l.strip()
                #print(l)
                if not l or l[0] == '/':
                    break
                if '//' in l or '/*' in l:
                    # Strip trailing comment.  Does not handle multi-line /*..*/.
                    comment = '//' if '//' in l else '/*'
                    pos = l.index(comment)
                    l = l[:pos].strip()
                    continue
                if l[0] in '[],':
                    # Update brace level or ignore comma after brace block.
                    bracelevel += [1, -1, 0]['[],'.index(l[0])]
                    l = l[1:]
                    continue
                if '[' in l or ']' in l:
                    # Parse up until next brace.
                    pos = 999
                    for b in '[]':
                        if b in l:
                            pos = min(pos, l.index(b))
                    payload = l[:pos].strip()
                    l = l[pos:]
                else:
                    payload = l.strip()
                    l = ''
                if bracelevel == 2 and payload and payload[0] != '/':
                    radix = 16 if '0x' in payload else 10
                    payload_vals = [int(v, radix) for v in payload.split(',') if v]
                    vals.append(payload_vals)
            if len(vals) and bracelevel == 0:
                break
    return vals

sfvals = parse_simple_font_file('/Users/dpwe/Downloads/HD44780_eleif.h')
print(len(sfvals))
c = 0x32 - 0x20
print(sfvals[c])
show_bitmap(sfvals[c], 8)


192
[14, 17, 1, 2, 4, 8, 31]
    *** 
   *   *
       *
      * 
     *  
    *   
   *****


In [139]:
def write_gfx_font_file(path, name, vals, first_val=32, y_advance=8):
    """Write a multi-part font header file for Adafruit GFX.  vals[0] is bytes, vals[1][c] is charinfo."""
    with open(path, 'wt') as f:
        f.write('#pragma once\n')
        f.write('#include <Adafruit_GFX.h>\n')
        f.write('\n')
        f.write('const uint8_t {:s}Bitmaps[] PROGMEM = {{\n'.format(name))
        num_this_line = 0
        num_lines = 0
        f.write('   ')
        for byte in vals[0]:
            if num_lines or num_this_line:
                f.write(',')
            if num_this_line == 12:
                f.write('\n   ')
                num_this_line = 0
                num_lines += 1
            f.write(' 0x{:02x}'.format(byte))
            num_this_line += 1
        f.write('};\n')
        # Write the glyphinfo.
        f.write('const GFXglyph {:s}Glyphs[] PROGMEM = {{\n'.format(name))
        for glyphinfo in vals[1]:
            f.write('  {')
            f.write(', '.join(str(d) for d in glyphinfo))
            f.write('},\n')
        f.write('};\n')
        # Suffix
        last_val = first_val + len(vals[1])
        f.write('const GFXfont {:s} PROGMEM = {{(uint8_t *){:s}Bitmaps,\n'.format(name, name))
        f.write('                      (GFXglyph *){:s}Glyphs, 0x{:x},\n'.format(name, first_val))
        f.write('                      0x{:x}, {:d}}};\n'.format(last_val, y_advance))


In [144]:
def sfvals_to_gfxvals(sfvals, width=8):
    """Convert the raw simple_font_vals to a gfx_vals list"""
    # sfvals is a list of lists of bytes, giving rows of 8-bit-wide fonts.
    # gfx_vals[0] is a packed byte blob, gfx_vals[1] is a list of {offset, width, height, advance cursor, x offset, y offset} lists
    bytes = []
    offsets = []
    height = len(sfvals[0])
    for char_bytes in sfvals:
        assert len(char_bytes) == height
        offsets.append(len(bytes))
        bytes.extend(pack_bits(char_bytes, width))
    fontinfo_list = [[offsets[i], width, height, width + 1, 0, 1 - height] for i in range(len(offsets))]
    return [bytes, fontinfo_list]

gfx_vals = sfvals_to_gfxvals(sfvals, width=5)

write_gfx_font_file('/Users/dpwe/Downloads/HD44780.h', 'HD44780', gfx_vals)